In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 505 ms (started: 2021-08-30 12:53:24 +08:00)


In [ ]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 #1 Aug 30, 2021'

#git push origin master
git push

In [2]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl

time: 3.41 s (started: 2021-08-30 12:58:46 +08:00)


# 子模块

研究项目倾向于测试针对同一数据集的不同方法。 这在带有继承的 Lightning 中很容易做到。

例如，假设我们现在想要训练一个自动编码器用作 MNIST 图像的特征提取器。 我们正在从 LitMNIST 模块扩展我们的自动编码器，该模块已经定义了所有数据加载。 Autoencoder 模型中唯一改变的是初始化、转发、训练、验证和测试步骤。

In [ ]:
class Encoder(torch.nn.Module):
    pass


class Decoder(torch.nn.Module):
    pass


class AutoEncoder(LitMNIST):
    def __init__(self):
        super().__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
        self.metric = MSE()

    def forward(self, x):
        return self.encoder(x)

    def training_step(self, batch, batch_idx):
        x, _ = batch

        representation = self.encoder(x)
        x_hat = self.decoder(representation)

        loss = self.metric(x, x_hat)
        return loss

    def validation_step(self, batch, batch_idx):
        self._shared_eval(batch, batch_idx, "val")

    def test_step(self, batch, batch_idx):
        self._shared_eval(batch, batch_idx, "test")

    def _shared_eval(self, batch, batch_idx, prefix):
        x, _ = batch
        representation = self.encoder(x)
        x_hat = self.decoder(representation)

        loss = self.metric(x, x_hat)
        self.log(f"{prefix}_loss", loss)

我们可以使用同一个训练器来训练它

In [ ]:
autoencoder = AutoEncoder()
trainer = Trainer()
trainer.fit(autoencoder)

请记住，forward 方法应该定义 LightningModule 的实际用途。 在这种情况下，我们想使用 AutoEncoder 来提取图像表示

In [ ]:
some_images = torch.Tensor(32, 1, 28, 28)
representations = autoencoder(some_images)

# 调试

以下是使调试更容易的标志。

## fast_dev_run

该标志通过运行 n 如果设置为 n (int) 否则为 1 如果设置为 True 训练和验证批次来运行“单元测试”。 关键是检测训练/验证循环中的任何错误，而不必等待完整的 epoch 崩溃。

（参见：Trainer 的 fast_dev_run 参数）

In [ ]:
# 运行 1 个训练、验证、测试批次和程序结束
trainer = Trainer(fast_dev_run=True)

# 运行 7 个训练、验证、测试批次和程序结束
trainer = Trainer(fast_dev_run=7)

> 此参数将禁用调谐器、检查点回调、提前停止回调、记录器和记录器回调（如 LearningRateLogger）并且仅运行 1 个时期。

## 检查梯度范数

日志（到记录器），每个权重矩阵的范数。

（参见：Trainer 的 track_grad_norm 参数）

In [ ]:
# the 2-norm
trainer = Trainer(track_grad_norm=2)

## 记录 GPU 使用情况

记录（到记录器）主机上每个 GPU 的 GPU 使用情况。

（参见：Trainer 的 log_gpu_memory 参数）

In [ ]:
trainer = Trainer(log_gpu_memory=True)

## 使模型过拟合数据子集

一个好的调试技术是取一小部分数据（比如每个类 2 个样本），并尝试让您的模型过度拟合。 如果不能，则表明它不适用于大型数据集。

（参见：Trainer 的 overfit_batches 参数）

In [ ]:
# 仅使用 1% 的训练数据（并在 val 和 test 中使用相同的训练数据加载器（关闭 shuffle））
trainer = Trainer(overfit_batches=0.01)

# 类似，但无论数据集大小如何，都有固定的 10 个批次
trainer = Trainer(overfit_batches=10)

有了这个标志，训练集、验证集和测试集都将是同一个训练集。 我们还将替换训练集中的采样器，为您关闭 shuffle。

## 打印 LightningModule 的摘要

每当 `.fit()` 函数被调用时，Trainer 将打印 LightningModule 的权重摘要。 默认情况下，它只打印顶级模块。 如果要显示网络中的所有子模块，请使用“完整”选项：

In [ ]:
trainer = Trainer(weights_summary="full")

您还可以通过在 LightningModule 中设置 `example_input_array` 属性来显示所有图层的中间输入和输出大小。 它将打印这样的表格

In [ ]:
  | Name  | Type        | Params | In sizes  | Out sizes
--------------------------------------------------------------
0 | net   | Sequential  | 132 K  | [10, 256] | [10, 512]
1 | net.0 | Linear      | 131 K  | [10, 256] | [10, 512]
2 | net.1 | BatchNorm1d | 1.0 K    | [10, 512] | [10, 512]

当您在 Trainer 上调用 .fit() 时。 这可以帮助您找到图层组合中的错误。
也可以看看：
* weights_summary参数
* ModelSummary

## 缩短Epoch

有时，仅使用一定百分比的训练、验证或测试数据（或一组批次）会很有帮助。 例如，您可以使用 20% 的训练集和 1% 的验证集。

在 Imagenet 等更大的数据集上，这可以帮助您比等待一个完整的 epoch 更快地调试或测试一些事情。

In [ ]:
# 仅使用 10% 的训练数据和 1% 的 val 数据
trainer = Trainer(limit_train_batches=0.1, limit_val_batches=0.01)

# 使用 10 批 train 和 5 批 val
trainer = Trainer(limit_train_batches=10, limit_val_batches=5)

## 设置验证健全性步骤的数量

Lightning 在训练开始时运行了几个验证步骤。 这可以避免在验证循环中崩溃，进入冗长的训练循环。

（参见：训练师的 num_sanity_val_steps 参数）

In [ ]:
# DEFAULT
trainer = Trainer(num_sanity_val_steps=2)

# Early stopping

## 提前停止一个Epoch

您可以通过覆盖 `on_train_batch_start()` 在满足某些条件时返回 -1 来提前停止一个纪元。

如果您重复执行此操作，对于您最初请求的每个 epoch，那么这将停止您的整个运行。

## 使用 EarlyStopping 回调基于指标提前停止

EarlyStopping 回调可用于监控验证指标并在未观察到改进时停止训练。

要启用它：
* 导入 EarlyStopping 回调。
* 使用 log() 方法记录要监控的指标。
* 初始化回调，并将监视器设置为您选择的记录指标。
* 将 EarlyStopping 回调传递给 Trainer 回调标志。

In [ ]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping


def validation_step(self):
    self.log("val_loss", loss)


trainer = Trainer(callbacks=[EarlyStopping(monitor="val_loss")])

您可以通过更改其参数来自定义回调行为。

In [ ]:
early_stop_callback = EarlyStopping(monitor="val_accuracy", min_delta=0.00, patience=3, verbose=False, mode="max")
trainer = Trainer(callbacks=[early_stop_callback])

在极端点停止训练的其他参数：
* stop_threshold：一旦监控的数量达到这个阈值，立即停止训练。 当我们知道超出某个最佳值不会进一步使我们受益时，这很有用。
* divergence_threshold：一旦监控的数量变得比这个阈值更差，就停止训练。 当达到如此糟糕的值时，我们认为模型无法再恢复，最好提前停止并在不同的初始条件下运行。
* check_finite：开启后，如果监控指标变为 NaN 或无穷大，我们将停止训练。

如果您需要在训练的不同部分提前停止，请子类 EarlyStopping 并更改它的调用位置：

In [ ]:
class MyEarlyStopping(EarlyStopping):
    def on_validation_end(self, trainer, pl_module):
        # 覆盖它以在 val 循环结束时禁用提前停止
        pass

    def on_train_end(self, trainer, pl_module):
        # i相反，在训练循环结束时进行
        self._run_early_stopping_check(trainer, pl_module)

> EarlyStopping 回调在每个验证时期结束时运行，在默认配置下，在每个训练时期之后发生。 但是，可以通过在 Trainer 中设置各种参数来修改验证频率，例如 check_val_every_n_epoch 和 val_check_interval。 必须注意，耐心参数计算没有改进的验证时期的数量，而不是训练时期的数量。 因此，在参数 check_val_every_n_epoch=10 和pattern=3 的情况下，训练器在停止之前将执行至少 40 个训练 epoch。